In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import matplotlib.pyplot as plt
import math
import cv2
import random

In [ ]:
plt.figure(figsize=(28,4))
rgb_lena_img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/cv_2024_1/lena.bmp', cv2.IMREAD_COLOR)
rgb_lena_img = cv2.cvtColor(rgb_lena_img, cv2.COLOR_BGR2RGB)
plt.subplot(1,7,1); plt.imshow(rgb_lena_img); plt.title('rgb_lena_img')

rgb_pepper = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Image_Processing_2025_1/Image/pepper.bmp', cv2.IMREAD_COLOR)
rgb_pepper = cv2.cvtColor(rgb_pepper, cv2.COLOR_BGR2RGB)
plt.subplot(1,7,2); plt.imshow(rgb_pepper); plt.title('rgb_pepper')

rgb_landscape_forest_path = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Image_Processing_2025_1/Image/Landscape-Forest Path.png', cv2.IMREAD_COLOR)
rgb_landscape_forest_path = cv2.cvtColor(rgb_landscape_forest_path, cv2.COLOR_BGR2RGB)
plt.subplot(1,7,3); plt.imshow(rgb_landscape_forest_path); plt.title('rgb_landscape_forest_path')

rgb_landscape_rocky_stream = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Image_Processing_2025_1/Image/Landscape-Rocky Stream.png',cv2.IMREAD_COLOR)
rgb_landscape_rocky_stream = cv2.cvtColor(rgb_landscape_rocky_stream, cv2.COLOR_BGR2RGB)
plt.subplot(1,7,4); plt.imshow(rgb_landscape_rocky_stream); plt.title('rgb_landscape_rocky_stream')

rgb_landscape_snowy_mountains = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Image_Processing_2025_1/Image/Landscape-Snowy Mountains.png',cv2.IMREAD_COLOR)
rgb_landscape_snowy_mountains = cv2.cvtColor(rgb_landscape_snowy_mountains, cv2.COLOR_BGR2RGB)
plt.subplot(1,7,5); plt.imshow(rgb_landscape_snowy_mountains); plt.title('rgb_landscape_snowy_mountains')

rgb_landscape_springtime_path = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Image_Processing_2025_1/Image/Landscape_Springtime Path.png',cv2.IMREAD_COLOR)
rgb_landscape_springtime_path = cv2.cvtColor(rgb_landscape_springtime_path, cv2.COLOR_BGR2RGB)
plt.subplot(1,7,6); plt.imshow(rgb_landscape_springtime_path); plt.title('rgb_landscape_springtime_path')

rgb_landscape_sunriese_lake = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Image_Processing_2025_1/Image/Landscape_Sunrise Lake.png',cv2.IMREAD_COLOR)
rgb_landscape_sunriese_lake = cv2.cvtColor(rgb_landscape_sunriese_lake, cv2.COLOR_BGR2RGB)
plt.subplot(1,7,7); plt.imshow(rgb_landscape_sunriese_lake); plt.title('rgb_landscape_sunriese_lake')


In [ ]:
def NN_interpolation(img, target_shape):
    if len(img.shape) == 2:  # 흑백 이미지
        h, w = img.shape
        new_h, new_w = target_shape
        output = np.zeros(target_shape, dtype=img.dtype)
    else:  # RGB 이미지
        h, w, channels = img.shape
        new_h, new_w = target_shape
        output = np.zeros((new_h, new_w, channels), dtype=img.dtype)

    # 출력 이미지의 크기에 대한 입력 이미지의 비율
    scale_x = w / new_w
    scale_y = h / new_h

    for i in range(new_h):  # i: y축
        for j in range(new_w):  # j: x축
            # 가장 가까운 픽셀의 좌표 계산
            x = min(int(j * scale_x), w - 1)  # 경계 처리
            y = min(int(i * scale_y), h - 1)
            if len(img.shape) == 2:  # 흑백 이미지
                output[i, j] = img[y, x]
            else:  # RGB 이미지
                output[i, j, :] = img[y, x, :]  # 모든 채널 복사

    return np.clip(output, 0, 255).astype(np.uint8)


# Bicubic interpolation (컬러 & 흑백)
def bicubic_interpolation(img, target_shape):
    if len(img.shape) == 3:  # 컬러 이미지일 경우
        h, w, c = img.shape
        new_h, new_w = target_shape
        output = np.zeros((new_h, new_w, c), dtype=img.dtype)
        for channel in range(c):
            scale_x = w / new_w
            scale_y = h / new_h
            for i in range(new_h):
                for j in range(new_w):
                    x = j * scale_x
                    y = i * scale_y
                    x0 = int(x)
                    y0 = int(y)
                    # Bicubic interpolation을 위해 주변 4x4 픽셀 사용
                    x_indices = [max(x0 - 1, 0), x0, min(x0 + 1, w - 1), min(x0 + 2, w - 1)]
                    y_indices = [max(y0 - 1, 0), y0, min(y0 + 1, h - 1), min(y0 + 2, h - 1)]
                    dx = x - x0
                    dy = y - y0
                    # Bicubic 커널 계산
                    def cubic(t):
                        t = np.abs(t)
                        if t <= 1:
                            return 1 - 2*t**2 + t**3
                        elif t <= 2:
                            return -t**3 + 5*t**2 - 8*t + 4
                        return 0
                    value = 0
                    for m in range(4):
                        for n in range(4):
                            weight = cubic(dx - (m - 1)) * cubic(dy - (n - 1))
                            value += weight * img[y_indices[n], x_indices[m], channel]
                    output[i, j, channel] = np.clip(value, 0, 255)
    else:  # 흑백 이미지일 경우
        h, w = img.shape
        new_h, new_w = target_shape
        output = np.zeros(target_shape, dtype=img.dtype)
        scale_x = w / new_w
        scale_y = h / new_h
        for i in range(new_h):
            for j in range(new_w):
                x = j * scale_x
                y = i * scale_y
                x0 = int(x)
                y0 = int(y)
                x_indices = [max(x0 - 1, 0), x0, min(x0 + 1, w - 1), min(x0 + 2, w - 1)]
                y_indices = [max(y0 - 1, 0), y0, min(y0 + 1, h - 1), min(y0 + 2, h - 1)]
                dx = x - x0
                dy = y - y0
                def cubic(t):
                    t = np.abs(t)
                    if t <= 1:
                        return 1 - 2*t**2 + t**3
                    elif t <= 2:
                        return -t**3 + 5*t**2 - 8*t + 4
                    return 0
                value = 0
                for m in range(4):
                    for n in range(4):
                        weight = cubic(dx - (m - 1)) * cubic(dy - (n - 1))
                        value += weight * img[y_indices[n], x_indices[m]]
                output[i, j] = np.clip(value, 0, 255)
    return np.clip(output, 0, 255).astype(np.uint8)


In [ ]:
# 공통으로 사용하는 함수
# PSNR 계산 함수-(height, width, channels) 형태의 RGB 이미지
def psnr_func(image1, image2):
    mse = np.mean((image1 - image2) ** 2)  # 모든 채널에 대해 MSE 계산
    psnr = 20 * math.log10(255 / math.sqrt(mse))  # PSNR 계산
    return psnr

# 가우시안 필터 생성 함수
def gaussian_filter(k, sig):
    ax = np.linspace(-(k // 2), k // 2, k)
    x, y = np.meshgrid(ax, ax)
    kernel = np.exp(-(x ** 2 + y ** 2) / (2 * sig ** 2))  # k x k 크기의 커널
    return kernel / np.sum(kernel)  # 정규화

# 가우시안 필터 마스크
gaussian_filter_mask = gaussian_filter(5, 1)

# 컨볼루션 함수
def convolution(image, filter):
    if len(image.shape) == 2:  # 흑백 이미지
        image_height, image_width = image.shape
        filtered_image = np.zeros_like(image)
    else:  # RGB 이미지
        image_height, image_width, channels = image.shape
        filtered_image = np.zeros_like(image)

    filter_height, filter_width = filter.shape
    padding = filter_height // 2

    # RGB 이미지의 경우 각 채널에 대해 컨볼루션 수행
    for y in range(padding, image_height - padding):
        for x in range(padding, image_width - padding):
            if len(image.shape) == 2:  # 흑백 이미지
                image_patch = image[y - padding:y + padding + 1, x - padding:x + padding + 1]
                filtered_image[y, x] = np.sum(image_patch * filter)
            else:  # RGB 이미지
                for c in range(channels):
                    image_patch = image[y - padding:y + padding + 1, x - padding:x + padding + 1, c]
                    filtered_image[y, x, c] = np.sum(image_patch * filter)

    return np.clip(filtered_image, 0, 255).astype(np.uint8)

# 가우시안 피라미드 생성 함수
def gaussian_pyramid(image, scale_num, levels):
    pyramid_img = [image]
    downsample_num = int(1 // scale_num)

    for _ in range(levels - 1):
        blur_img = convolution(pyramid_img[-1], gaussian_filter_mask)  # 각 채널에 대해 블러링
        # 다운샘플링
        if len(image.shape) == 2:  # 흑백 이미지
            downsample_img = blur_img[::downsample_num, ::downsample_num]
        else:  # RGB 이미지--차원수 다름
            downsample_img = blur_img[::downsample_num, ::downsample_num, :]
        pyramid_img.append(downsample_img)

    return pyramid_img  # 다운샘플된 이미지 리스트 반환

# 업샘플링 이후 가우시안 스무딩 적용
def upsample(img, target_shape):
    # 입력 이미지와 목표 크기의 비율 계산
    scale_x = target_shape[0] // img.shape[0]
    scale_y = target_shape[1] // img.shape[1]
    interpolated_img = bicubic_interpolation(img, (scale_x * img.shape[0], scale_y * img.shape[1]))
    # 계단 현상 완화를 위한 가우시안 필터링 적용
    final_upsampled_img = convolution(interpolated_img, gaussian_filter_mask)
    return final_upsampled_img

# 3가지 해상도의 이미지를 결합하여 최종 이미지(512x512) 구성
def reconstruct_image(img_128, img_256, img_512):
    up_128 = upsample(img_128, (512, 512))  # 128 --> 512 업샘플링
    up_256 = upsample(img_256, (512, 512))  # 256 --> 512 업샘플링
    # 업샘플링된 이미지를 가중치를 적용해서 하나로 결합
    reconstructed_img = 0.4 * up_128 + 0.3 * up_256 + 0.3 * img_512
    return np.clip(reconstructed_img, 0, 255)  # 픽셀값 0~255로 제한

def random_region_selection(img, k):
    patch_size = k
    num_patches = 16384 // (patch_size**2)

    if len(img.shape) == 2:  # 흑백 이미지
        output = np.zeros_like(img)
        h, w = img.shape
    else:  # RGB 이미지
        output = np.zeros_like(img)
        h, w, channels = img.shape

    np.random.seed(42)
    for _ in range(num_patches):
        # 패치가 이미지 경계를 넘지 않도록 좌상단 좌표 무작위 선정
        x = np.random.randint(0, h - patch_size)
        y = np.random.randint(0, w - patch_size)
        # 패치 복사 (흑백 또는 RGB)
        output[x:x+patch_size, y:y+patch_size] = img[x:x+patch_size, y:y+patch_size]

    return output


In [ ]:
# 1번 이미지
gaus_pyramid_img = gaussian_pyramid(rgb_pepper, 0.5, 3)  # 0.5비율로 scaling, 3계층
img_512 = gaus_pyramid_img[0]  # 512x512x3
img_256 = gaus_pyramid_img[1]  # 256x256x3
img_128 = gaus_pyramid_img[2]  # 128x128x3


In [ ]:
# 랜덤기반 영역 추출
img_256_sparse = random_region_selection(img_256, 16)
img_512_sparse = random_region_selection(img_512, 16)

# 이미지 재구성
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)

# psnr 결과
score = psnr_func(rgb_pepper, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img_128); plt.title('img_128')
plt.subplot(1,3,2); plt.imshow(img_256_sparse); plt.title('img_256_sparse')
plt.subplot(1,3,3); plt.imshow(img_512_sparse); plt.title('img_512_sparse')
plt.show()
# Optional img vs Reconstructed img
img_reconstructed = img_reconstructed.astype(np.uint8)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(rgb_pepper)
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed)
plt.show()


In [ ]:
# 2번 이미지
gaus_pyramid_img = gaussian_pyramid(rgb_landscape_forest_path, 0.5, 3)  # 0.5비율로 scaling, 3계층
img_512 = gaus_pyramid_img[0]  # 512x512x3
img_256 = gaus_pyramid_img[1]  # 256x256x3
img_128 = gaus_pyramid_img[2]  # 128x128x3


In [ ]:
# 랜덤기반 영역 추출
img_256_sparse = random_region_selection(img_256, 16)
img_512_sparse = random_region_selection(img_512, 16)

# 이미지 재구성
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)

# psnr 결과
score = psnr_func(rgb_landscape_forest_path, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img_128); plt.title('img_128')
plt.subplot(1,3,2); plt.imshow(img_256_sparse); plt.title('img_256_sparse')
plt.subplot(1,3,3); plt.imshow(img_512_sparse); plt.title('img_512_sparse')
plt.show()
# Optional img vs Reconstructed img
img_reconstructed = img_reconstructed.astype(np.uint8)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(rgb_landscape_forest_path)
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed)
plt.show()


In [ ]:
# 3번 이미지
gaus_pyramid_img = gaussian_pyramid(rgb_landscape_rocky_stream, 0.5, 3)  # 0.5비율로 scaling, 3계층
img_512 = gaus_pyramid_img[0]  # 512x512x3
img_256 = gaus_pyramid_img[1]  # 256x256x3
img_128 = gaus_pyramid_img[2]  # 128x128x3



In [ ]:
# 랜덤기반 영역 추출
img_256_sparse = random_region_selection(img_256, 32)
img_512_sparse = random_region_selection(img_512, 32)

# 이미지 재구성
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)

# psnr 결과
score = psnr_func(rgb_landscape_rocky_stream, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img_128); plt.title('img_128')
plt.subplot(1,3,2); plt.imshow(img_256_sparse); plt.title('img_256_sparse')
plt.subplot(1,3,3); plt.imshow(img_512_sparse); plt.title('img_512_sparse')
plt.show()
# Optional img vs Reconstructed img
img_reconstructed = img_reconstructed.astype(np.uint8)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(rgb_landscape_rocky_stream)
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed)
plt.show()


In [ ]:
# 4번 이미지
gaus_pyramid_img = gaussian_pyramid(rgb_landscape_snowy_mountains, 0.5, 3)  # 0.5비율로 scaling, 3계층
img_512 = gaus_pyramid_img[0]  # 512x512x3
img_256 = gaus_pyramid_img[1]  # 256x256x3
img_128 = gaus_pyramid_img[2]  # 128x128x3


In [ ]:
# 랜덤기반 영역 추출
img_256_sparse = random_region_selection(img_256, 32)
img_512_sparse = random_region_selection(img_512, 32)

# 이미지 재구성
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)

# psnr 결과
score = psnr_func(rgb_landscape_snowy_mountains, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img_128); plt.title('img_128')
plt.subplot(1,3,2); plt.imshow(img_256_sparse); plt.title('img_256_sparse')
plt.subplot(1,3,3); plt.imshow(img_512_sparse); plt.title('img_512_sparse')
plt.show()
# Optional img vs Reconstructed img
img_reconstructed = img_reconstructed.astype(np.uint8)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(rgb_landscape_snowy_mountains)
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed)
plt.show()


In [ ]:
# 5번 이미지
gaus_pyramid_img = gaussian_pyramid(rgb_landscape_springtime_path, 0.5, 3)  # 0.5비율로 scaling, 3계층
img_512 = gaus_pyramid_img[0]  # 512x512x3
img_256 = gaus_pyramid_img[1]  # 256x256x3
img_128 = gaus_pyramid_img[2]  # 128x128x3


In [ ]:
# 랜덤기반 영역 추출
img_256_sparse = random_region_selection(img_256, 32)
img_512_sparse = random_region_selection(img_512, 32)

# 이미지 재구성
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)

# psnr 결과
score = psnr_func(rgb_landscape_springtime_path, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img_128); plt.title('img_128')
plt.subplot(1,3,2); plt.imshow(img_256_sparse); plt.title('img_256_sparse')
plt.subplot(1,3,3); plt.imshow(img_512_sparse); plt.title('img_512_sparse')
plt.show()
# Optional img vs Reconstructed img
img_reconstructed = img_reconstructed.astype(np.uint8)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(rgb_landscape_springtime_path)
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed)
plt.show()


In [ ]:
# 6번 이미지
gaus_pyramid_img = gaussian_pyramid(rgb_landscape_sunriese_lake, 0.5, 3)  # 0.5비율로 scaling, 3계층
img_512 = gaus_pyramid_img[0]  # 512x512x3
img_256 = gaus_pyramid_img[1]  # 256x256x3
img_128 = gaus_pyramid_img[2]  # 128x128x3


In [ ]:
# 랜덤기반 영역 추출
img_256_sparse = random_region_selection(img_256, 32)
img_512_sparse = random_region_selection(img_512, 32)

# 이미지 재구성
img_reconstructed = reconstruct_image(img_128, img_256_sparse, img_512_sparse)

# psnr 결과
score = psnr_func(rgb_landscape_sunriese_lake, img_reconstructed)
print(f"PSNR of reconstructed image: {score:.4f}")

plt.figure(figsize=(15,5))
plt.subplot(1,3,1); plt.imshow(img_128); plt.title('img_128')
plt.subplot(1,3,2); plt.imshow(img_256_sparse); plt.title('img_256_sparse')
plt.subplot(1,3,3); plt.imshow(img_512_sparse); plt.title('img_512_sparse')
plt.show()
# Optional img vs Reconstructed img
img_reconstructed = img_reconstructed.astype(np.uint8)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1); plt.title("Original img"); plt.imshow(rgb_landscape_sunriese_lake)
plt.subplot(1,2,2); plt.title("Reconstructed img"); plt.imshow(img_reconstructed)
plt.show()
